In [2]:
from IPython.display import display, clear_output
from ipywidgets import widgets, Button, Layout
import sympy as sym
import time

def create_matrix_B1(n, R, q, f):
    f = R(f)
    x = R.gen()
    a = R(1)
    B = [0] * n
    for i in range(n):
        B[i] = [0] * n
    
    for i in range(n):
        for j in range(n):
            B[j][i] = a[j]
            if i == j:
                B[i][i] -= 1
        a = (a*x**q)%f
    return Matrix(GF(q), B)

def test(q,f_x):
    p = q
    polynoms=[]
    F = GF(q)['x']
    #x = var('x')
    x = sym.Symbol('x')
    
    while 1:
        j=0
        f_tmp=0
        while j<=f_x.degree():
            f_tmp+=int(f_x[j])*x^j
            j+=1 
        if F(diff(f_tmp,x))==0:
            f_tmp=f_tmp.substitute(x = x^(1/p))
            coef=f_tmp.coefficient(x^(1/p),1)
            if coef>0:
                f_tmp-=coef*x^(1/p)
                f_tmp+=coef*x
            flag=1
        if F(diff(f_tmp,x))!=1:
            g=R.gcd(f_tmp,R(diff(f_tmp,x)))
            if g!=1:
                
                f_tmp=R(f_tmp)//g
                if f_tmp not in polynoms:
                    polynoms.append(f_tmp)
                f_x=F(g)
                continue
            else:
                
                g=1
                f_tmp=R(f_tmp)//g
                if f_tmp not in polynoms:
                    polynoms.append(f_tmp)
                break
        else:
            g=1
            f_tmp=R(f_tmp)//g
            if f_tmp not in polynoms:
                polynoms.append(f_tmp)
            break
    return f_tmp,polynoms


def Berlikamp(q, f,R):
    f_der = f.derivative()
    f,polynoms = test(q,f)
    f = R(f)
    f_der = f.derivative()
    U = []
    for polynom in polynoms:
        U_temp = Berlikamp_dop(q, polynom,R)
        if len(U_temp) == 0:                
            U.append(polynom)
        else:
            U.extend(U_temp)
    return set(U)



def Berlikamp_dop(q, f,R):
    U_prev = []
    f_der = f.derivative()
   
    d = gcd(f, f_der)
    if d.degree() != 0:
        f = R(f/d)
    n = f.degree()

    B1 = create_matrix_B1(n, R, q, f)
    if B1.rank() == n - 1:
#         print('Многочлен неприводимый')
        return [f]

    e = B1.right_kernel().basis_matrix().rows()
    
    r = len(e)
    if r == 0:
        return []
    i = 1
    l = 0
    U = [R(0)]*r
    U[l] = f

    while l != (r - 1) and i != r:

        h = R(0)
        for j in range(n):
            h += e[i][j]*x**j

        for j in range(l + 1):

            s = 0
            gx = []
            for c in range(q):
                d = gcd(U[j], h - R(c))
                if (d != R(1)) and (d not in gx):
                    gx.append(d)
                    s += 1

            U[j] = gx[0]
            t = 1
            while t < s:
                U[l + t] = gx[t]
                t += 1
            l = l+s-1

            if l == r - 1:
                break

        i += 1
    return set(U)



def Result(result,f_const,R):
    count=[]
    result.discard(1)
    result = list(result)
    
    i=0
    for res in result:
        count.append(0)
        f_const = R(f_const)
        res = R(res)
        while gcd(f_const,res)!=1:
            f_const=f_const/res
            count[i]+=1
        if count[i]!=1:
            if len(str(result[i]))!=1:
                result[i]='('+str(result[i])+')^'+str(count[i])
            else:
                result[i]=str(result[i])+'^'+str(count[i])
        else:
            result[i]=str(result[i])
        i+=1
    print('Результат разложения: '+str(result)+'\n')
    
q = int(input("Введите q = "))
#print("q:", q)
R.<x> = PolynomialRing(Zmod(q))


polynomial = input("Введите полином: ")
f = R(polynomial)

output1 = widgets.Output(layout={'border': '1px solid #000'})
output2 = widgets.Output(layout={'border': '1px solid #000'})
output3 = widgets.Output(layout={'border': '1px solid #000'})

def display_result(q, f):
    with output3:
        clear_output()  # Очистить предыдущий вывод
        print('Введенный полином = ', f)
        print('q = ', q)
        
def display_result_U(U,Time_spent):
    with output1:
        clear_output()  # Очистить предыдущий вывод
        Result(U, f, R)
        print('Время выполнения = ', Time_spent)
        
def display_result_Sage(f):
    with output2:
        clear_output()
        print("Sage:")
        if is_prime(q):
            print(f.factor())
        else:
            print('Sage не поддерживает разложение многочленов над такими кольцами.')

# Создаем кнопку с нужными параметрами
button1 = Button(description="Алгоритм Берлекэмпа", 
                        button_style='success',layout=Layout(width='20%', height='50px') # 'success', 'info', 'warning', 'danger', ''
                        )
display_result(q,f)
def on_button_clicked(b):
    start_time = time.time()
    U = Berlikamp(q, f, R)
    Time_spent =  time.time() - start_time
    # Выводим результат
    display_result_Sage(f)
    display_result_U(U, Time_spent)
   
    
button1.on_click(on_button_clicked) # Назначаем этот обработчик на событие "on_click"

display(output3, button1, output1, output2) # Отображаем кнопку

#q = 2050079 x^16 +x^12 + x^4 + 1
#x^18 + 2050014*x^17 + 1788*x^16 + 2024971*x^15 + 124879*x^14 + 1931169*x^13 +780409*x^12 + 1344201*x^11 + 607298*x^10 + 27932*x^9 + 48493*x^8 + 796104*x^7 + 1639985*x^6 + 1850770*x^5 + 1645530*x^4 + 1009691*x^3 + 1696631*x^2 + 222532*x + 273649


Введите q = 2050079
Введите полином: x^18 + 2050014*x^17 + 1788*x^16 + 2024971*x^15 + 124879*x^14 + 1931169*x^13 +780409*x^12 + 1344201*x^11 + 607298*x^10 + 27932*x^9 + 48493*x^8 + 796104*x^7 + 1639985*x^6 + 1850770*x^5 + 1645530*x^4 + 1009691*x^3 + 1696631*x^2 + 222532*x + 273649


Output(layout=Layout(border='1px solid #000'))

Button(button_style='success', description='Алгоритм Берлекэмпа', layout=Layout(height='50px', width='20%'), s…

Output(layout=Layout(border='1px solid #000'))

Output(layout=Layout(border='1px solid #000'))